#### Project: https://github.com/vwxyzjn/gym-microrts/blob/master/experiments/paper/ppo_diverse.py
Class: **ppo_diverse.py**  -> pytorch 
Codezeile: 262-352 -> **Agent** initialize 

In [ ]:
args.batch_size = int(args.num_envs * args.num_steps)
args.minibatch_size = int(args.batch_size // args.n_minibatch)

...

# flatten the batch
b_obs = obs.reshape((-1,) + envs.observation_space.shape)
b_logprobs = logprobs.reshape(-1)
b_actions = actions.reshape((-1,) + envs.action_space.shape)
b_advantages = advantages.reshape(-1)
b_returns = returns.reshape(-1)
b_values = values.reshape(-1)
b_invalid_action_masks = invalid_action_masks.reshape((-1, invalid_action_masks.shape[-1]))

# Optimizaing the policy and value network
target_agent = Agent().to(device)
inds = np.arange(
    args.batch_size,
)
for i_epoch_pi in range(args.update_epochs):
    np.random.shuffle(inds)
    for start in range(0, args.batch_size, args.minibatch_size):
        end = start + args.minibatch_size
        minibatch_ind = inds[start:end]
        mb_advantages = b_advantages[minibatch_ind]
        if args.norm_adv:
            mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + 1e-8)
        _, newlogproba, entropy, _, new_values = agent.get_action_and_value(
            b_obs[minibatch_ind], b_actions.long()[minibatch_ind], b_invalid_action_masks[minibatch_ind], envs, device
        )
        ratio = (newlogproba - b_logprobs[minibatch_ind]).exp()

        # Stats
        approx_kl = (b_logprobs[minibatch_ind] - newlogproba).mean()

        # Policy loss
        pg_loss1 = -mb_advantages * ratio
        pg_loss2 = -mb_advantages * torch.clamp(ratio, 1 - args.clip_coef, 1 + args.clip_coef)
        pg_loss = torch.max(pg_loss1, pg_loss2).mean()
        entropy_loss = entropy.mean()

        # Value loss
        new_values = new_values.view(-1)
        if args.clip_vloss:
            v_loss_unclipped = (new_values - b_returns[minibatch_ind]) ** 2
            v_clipped = b_values[minibatch_ind] + torch.clamp(
                new_values - b_values[minibatch_ind], -args.clip_coef, args.clip_coef
            )
            v_loss_clipped = (v_clipped - b_returns[minibatch_ind]) ** 2
            v_loss_max = torch.max(v_loss_unclipped, v_loss_clipped)
            v_loss = 0.5 * v_loss_max.mean()
        else:
            v_loss = 0.5 * ((new_values - b_returns[minibatch_ind]) ** 2)

        loss = pg_loss - args.ent_coef * entropy_loss + v_loss * args.vf_coef

        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(agent.parameters(), args.max_grad_norm)
        optimizer.step()

    if args.kle_stop:
         if approx_kl > args.target_kl:
              break
    if args.kle_rollback:
        if (
            b_logprobs[minibatch_ind]
            - agent.get_action(
                b_obs[minibatch_ind], b_actions.long()[minibatch_ind].T, b_invalid_action_masks[minibatch_ind], envs
             )[1]
        ).mean() > args.target_kl:
            agent.load_state_dict(target_agent.state_dict())
            break

...


Der folgende Codeabschnitt aus ppo_diverse.py beschreibt die Verwendung von PPO, wodurch der Agent eine Optimierung der Policy erhält. In folgenden wird das Training für K Epochen in der Codezeile 20 "for i_epoch_pi in range(args.update_epochs)" durchgeführt. Dabei werden die Daten und die daraus ausgewählte mini-batches(minibatch_ind) für das Training gesammelt und genutzt. Das Wahrscheinlichkeitsverhältnis **"ratio"** wird in der Codezeile 31 zwischen der alten Policy und der neuen Policy definiert. 

Ab der Zeile 36-40 wird die "Policy Loss"-Funktion definiert. Hierbei beschreibt **"pg_loss1"** den "unclipped objective"-Term mit der Berechnung des Wahrscheinlichkeitsverhältnis und des Advantages. **"pg_loss2"** beschreibt den "clipped Objective"-Term, in der das Wahrscheinlichkeitsverhältnis in einem Bereich  zwischen **1-𝜖** und **1+𝜖** abgeschnitten wird und auch mit der Advantages berechnet wird. Die „Clipped Surrogate Objective function“ wird in der Codezeile 39 dargestellt, welches das Wahrscheinlichkeitsverhältnis unter der neuen bzw. alten Policy zwischen einem bestimmten Bereich abschneidet und nicht zulässt, dass sie sich weiter von diesem Bereich entfernt.

Im nächsten Schritt wird die **"Value Loss"-Funktion** bzw. “Clipped Surrogate Objective Loss“-Funktion ab der Codezeile 43-53 berechnet, die hierbei den „squared-error value loss“ bzw. ein mittlerer quadratischer Fehler der Wertfunktion beschreibt. Hier wird versucht die Differenz zwischen dem geschätzten Wert und dem tatsächlichen Wert zu minimieren.

Die endgültige “Clipped Surrogate Objective Loss“-Funktion wird in der Codezeile 55 **"loss = pg_loss - args.ent_coef * entropy_loss + v_loss * args.vf_coef"** definiert. Die Variablen **"args.ent_coef"** und **"args.vf_coef"** beschreiben die Hyperparameter. Die Variable **"v_loss"** und **"entropy_loss"** sind die zusätzlichen Terme, die bei der endgültige “Clipped Surrogate Objective Loss“-Funktion ergänzt werden. **"v_loss"** definiert den Fehlerterm für die Wertschätzung und **"entropy_loss"** beschreibt einen Entropieterm, der für die ausreichende Exploration des Agenten fördert.





#### Project: https://github.com/vwxyzjn/gym-microrts/blob/master/experiments/ppo_gridnet.py

Class: ppo_gridnet.py

In [ ]:
args.batch_size = int(args.num_envs * args.num_steps)
args.minibatch_size = int(args.batch_size // args.n_minibatch)

...

# flatten the batch
b_obs = obs.reshape((-1,) + envs.observation_space.shape)
b_logprobs = logprobs.reshape(-1)
b_actions = actions.reshape((-1,) + action_space_shape)
b_advantages = advantages.reshape(-1)
b_returns = returns.reshape(-1)
b_values = values.reshape(-1)
b_invalid_action_masks = invalid_action_masks.reshape((-1,) + invalid_action_shape)

# Optimizaing the policy and value network
inds = np.arange(
    args.batch_size,
)
for i_epoch_pi in range(args.update_epochs):
    np.random.shuffle(inds)
    for start in range(0, args.batch_size, args.minibatch_size):
        end = start + args.minibatch_size
        minibatch_ind = inds[start:end]
        mb_advantages = b_advantages[minibatch_ind]
        if args.norm_adv:
            mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + 1e-8)
        _, newlogproba, entropy, _, new_values = agent.get_action_and_value(
            b_obs[minibatch_ind], b_actions.long()[minibatch_ind], b_invalid_action_masks[minibatch_ind], envs, device
        )
        ratio = (newlogproba - b_logprobs[minibatch_ind]).exp()

        # Stats
        approx_kl = (b_logprobs[minibatch_ind] - newlogproba).mean()

        # Policy loss
        pg_loss1 = -mb_advantages * ratio
        pg_loss2 = -mb_advantages * torch.clamp(ratio, 1 - args.clip_coef, 1 + args.clip_coef)
        pg_loss = torch.max(pg_loss1, pg_loss2).mean()
        entropy_loss = entropy.mean()

        # Value loss
        new_values = new_values.view(-1)
        if args.clip_vloss:
            v_loss_unclipped = (new_values - b_returns[minibatch_ind]) ** 2
            v_clipped = b_values[minibatch_ind] + torch.clamp(
                new_values - b_values[minibatch_ind], -args.clip_coef, args.clip_coef
            )
            v_loss_clipped = (v_clipped - b_returns[minibatch_ind]) ** 2
            v_loss_max = torch.max(v_loss_unclipped, v_loss_clipped)
            v_loss = 0.5 * v_loss_max.mean()
        else:
            v_loss = 0.5 * ((new_values - b_returns[minibatch_ind]) ** 2)

        loss = pg_loss - args.ent_coef * entropy_loss + v_loss * args.vf_coef

        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(agent.parameters(), args.max_grad_norm)
        optimizer.step()

#### Project: https://github.com/openai/baselines/blob/9fa8e1baf1d1f975b87b369a8082122eac812eb1/baselines/ppo1/pposgd_simple.py#L111-L117

class: ppo1/pposgd_simple.py   mit Tensorflow
Baseline Implementierung von PPO von OpenAI

In [ ]:
    ratio = tf.exp(pi.pd.logp(ac) - oldpi.pd.logp(ac)) # pnew / pold
    surr1 = ratio * atarg # surrogate from conservative policy iteration
    surr2 = tf.clip_by_value(ratio, 1.0 - clip_param, 1.0 + clip_param) * atarg #
    pol_surr = - tf.reduce_mean(tf.minimum(surr1, surr2)) # PPO's pessimistic surrogate (L^CLIP)
    vf_loss = tf.reduce_mean(tf.square(pi.vpred - ret))
    total_loss = pol_surr + pol_entpen + vf_loss
    losses = [pol_surr, pol_entpen, vf_loss, meankl, meanent]

Der folgende Codeabschnitt aus der Baseline von **pposgd_simple.py**: 

**ratio** -> Wahrscheinlichkeitsverhältnis zwischen der neuen und alten Policy

**surr1** -> Unclipped Objective 

**surr2** -> Clipped Objective, Beschneidung zwischen 1-𝜖 und 1+𝜖

**pol_surr** -> „Clipped Surrogate Objective"-Funktion (Clipping Mechanismus von PPO)

**vf_loss** -> Beschreibt den „squared-error value loss“ bzw. ein mittlerer quadratischer Fehler der Wertfunktion

**total_loss** -> Endgültige “Clipped Surrogate Objective Loss“-Funktion, welches für das Trainieren eines Agenten genutzt wird. 

#### Project: https://github.com/higgsfield/RL-Adventure-2/blob/master/3.ppo.ipynb

Class:RL-Adventure-2/3.ppo.ipynb -> mit pytorch

In [ ]:
def ppo_iter(mini_batch_size, states, actions, log_probs, returns, advantage):
    batch_size = states.size(0)
    for _ in range(batch_size // mini_batch_size):
        rand_ids = np.random.randint(0, batch_size, mini_batch_size)
        yield states[rand_ids, :], actions[rand_ids, :], log_probs[rand_ids, :], returns[rand_ids, :], advantage[rand_ids, :]
        
        

def ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantages, clip_param=0.2):
    for _ in range(ppo_epochs):
        for state, action, old_log_probs, return_, advantage in ppo_iter(mini_batch_size, states, actions, log_probs, returns, advantages):
            dist, value = model(state)
            entropy = dist.entropy().mean()
            new_log_probs = dist.log_prob(action)

            ratio = (new_log_probs - old_log_probs).exp()
            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1.0 - clip_param, 1.0 + clip_param) * advantage

            actor_loss  = - torch.min(surr1, surr2).mean()
            critic_loss = (return_ - value).pow(2).mean()

            loss = 0.5 * critic_loss + actor_loss - 0.001 * entropy

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

Der folgende Codeabschnitt mit der Funktionen **"ppo_iter"** und **"ppo_update"** beschreiben die mehrere Epochen für die Aktualisierung der Policy und das Clipping Mechanismus, welches die Trainingsstabilität verbessern soll, indem es die Änderungen an der Policy bei jedem Schritt begrenzt. 

In der Funktion "ppo_update" werden mehrere Epochen des Gradientenanstiegs für die "Samples" durchgeführt. Hierbei beschreibt **"ratio"** in der Codezeile 16 das Wahrscheinlichkeitsverhältnis (alten Policy und der neuen Policy).

Die Funktion **"actor_loss"** beschreibt die „Clipped Surrogate Objective"-Funktion, welches aus den Funktionen **"surr1"** (unclipped objective) und **"surr2"** (clipped objective) besteht.

Die Funktion **"critic_loss"** versucht die Differenz zwischen dem geschätzten Wert und dem tatsächlichen Wert zu minimieren ("squared-error value loss"-Funktion)

Die endgültige “Clipped Surrogate Objective Loss“ Funktion wird in der Codezeile 23 **"loss"** definiert, welches aus den Werten der “Clipped Surrogate Objective Loss“ Funktion(actor_loss) mit den zwei zusätzlichen Termen „squared-error value loss“(critic-loss) und Entropieterm (entropy) und deren Hyperparameter berechnet wird.

In [ ]:

num_inputs  = envs.observation_space.shape[0]
num_outputs = envs.action_space.shape[0]

#Hyper params:
hidden_size      = 256
lr               = 3e-4
num_steps        = 20
mini_batch_size  = 5
ppo_epochs       = 4
threshold_reward = -200

model = ActorCritic(num_inputs, num_outputs, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

Hier werden die Werte für die Hyperparameter definiert.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9ce75853-4372-4d76-b47c-34c3ad90a24b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>